In [10]:
import TensorFrost as tf
import numpy as np
import matplotlib.pyplot as plt
import time

tf.initialize(tf.cpu, "H:/cl_compile.bat /O1 /fp:fast /openmp")

S = 512

def matmul():
    A = tf.input([S, S], tf.float32)
    B = tf.input([S, S], tf.float32)

    N, M = A.shape
    K = B.shape[1]
    
    C = tf.zeros([N, K])

    i, j, k = tf.indices([N, K, M])

    tf.scatterAdd(C[i, j], A[i, k] * B[k, j])

    return [C]

mmul = tf.program(matmul)

In [11]:
Anp = np.random.rand(S, S).astype(np.float32)
Bnp = np.random.rand(S, S).astype(np.float32)

A = tf.memory(np.transpose(Anp))
B = tf.memory(np.transpose(Bnp))
C, = mmul(A, B)

Cnp = C.numpy

#compare to numpy
Cnp2 = np.dot(Bnp, Anp)

print(Cnp - Cnp2)


[[-7.6293945e-05  4.5776367e-05  3.0517578e-05 ... -3.0517578e-05
  -5.3405762e-05  1.5258789e-05]
 [-1.5258789e-05  4.5776367e-05 -3.8146973e-05 ... -1.5258789e-05
  -3.0517578e-05 -8.3923340e-05]
 [-4.5776367e-05 -3.0517578e-05  3.0517578e-05 ... -6.1035156e-05
  -1.5258789e-05 -6.1035156e-05]
 ...
 [-7.6293945e-05  7.6293945e-06  3.8146973e-05 ... -1.5258789e-05
   5.3405762e-05 -3.0517578e-05]
 [-1.5258789e-05 -3.0517578e-05 -2.2888184e-05 ...  0.0000000e+00
   1.5258789e-05  0.0000000e+00]
 [ 0.0000000e+00  4.5776367e-05 -1.5258789e-05 ... -1.5258789e-05
   1.5258789e-05 -4.5776367e-05]]


In [12]:
Anp = np.random.rand(S, S).astype(np.float32)
Bnp = np.random.rand(S, S).astype(np.float32)

A = tf.memory(np.transpose(Anp))
B = tf.memory(np.transpose(Bnp))

# Performance test
t0 = time.time()
for i in range(50):
    C, = mmul(A, B)
t1 = time.time()

tf_time = t1 - t0
print("TensorFrost: ", t1 - t0)

t0 = time.time()
for i in range(50):
    Cnp2 = np.dot(Bnp, Anp)
t1 = time.time()

np_time = t1 - t0
print("Numpy: ", t1 - t0)

print("TF/NP: ", tf_time / np_time)

TensorFrost:  14.598998069763184
Numpy:  0.1270008087158203
TF/NP:  114.95200871067057
